In [2]:
pip install deepsurv

  Using cached deepsurv-0.1.0.tar.gz (10 kB)
  Using cached Lasagne-0.1.tar.gz (125 kB)
  Using cached lifelines-0.27.1-py3-none-any.whl (349 kB)
  Using cached autograd-1.4-py3-none-any.whl (48 kB)
  Using cached autograd-gamma-0.5.0.tar.gz (4.0 kB)
  Using cached formulaic-0.3.4-py3-none-any.whl (68 kB)

  Using cached astor-0.8.1-py2.py3-none-any.whl (27 kB)
  Using cached interface_meta-1.3.0-py3-none-any.whl (14 kB)
  Created wheel for deepsurv: filename=deepsurv-0.1.0-py3-none-any.whl size=11164 sha256=34d6ee01c8de5f1d051bda0960423165c266ace783120cb69e758748b8b635a4
  Stored in directory: c:\users\james\appdata\local\pip\cache\wheels\8b\35\64\cb4fa77f7de25e6133c42cc1749ecd28b82131fcc937ab37ee
  Created wheel for lasagne: filename=Lasagne-0.1-py3-none-any.whl size=79281 sha256=05073c2e9b54266837b5c7a81dcdbd552663feb7ae08ddf431bfb8a538b74999
  Stored in directory: c:\users\james\appdata\local\pip\cache\wheels\25\5b\33\563d410432dfba1865b036703f71b81d8565b1393846d2e611
  Created whe

In [1]:
import pyarrow.feather as feather
import torchtuples as tt
import torch
import numpy as np
from pycox.models import CoxPH
import deepsurv

ModuleNotFoundError: No module named 'deepsurv'

In [61]:
cmprsk = feather.read_feather('melanoma.feather')

In [63]:
cmprsk ##scaled

,time,status,sex,age,year,thickness,ulcer
0,10.0,3.0,1.0,0.791209,0.666667,0.384527,1.0
1,30.0,3.0,1.0,0.571429,0.400000,0.031755,0.0
2,35.0,2.0,1.0,0.406593,1.000000,0.071594,0.0
3,99.0,3.0,0.0,0.736264,0.400000,0.161663,0.0
4,185.0,1.0,1.0,0.527473,0.200000,0.691686,1.0
...,...,...,...,...,...,...,...
200,4492.0,2.0,1.0,0.274725,0.200000,0.401848,1.0
201,4668.0,2.0,0.0,0.395604,0.200000,0.347575,0.0
202,4688.0,2.0,0.0,0.417582,0.200000,0.021940,0.0
203,4926.0,2.0,0.0,0.505495,0.133333,0.124711,0.0


In [75]:
df_train = cmprsk.copy()
df_test = df_train.sample(frac=0.2)
df_train = df_train.drop(df_test.index)
df_val = df_train.sample(frac=0.2)
df_train = df_train.drop(df_val.index)


x_train = df_train.astype('float32')
x_val = df_val.astype('float32')
x_test = df_test.astype('float32')

In [76]:
get_target = lambda df: (df['time'].values, df['status'].values)
y_train = get_target(x_train)
y_val = get_target(x_val)
durations_test, events_test = get_target(x_test)
val = x_val, y_val

In [77]:
# input settings for DeepSurv

n_nodes = 256
in_features = x_train.shape[1]
num_nodes = [n_nodes, n_nodes, n_nodes, n_nodes]
out_features = 1
batch_norm = True
dropout = 0.4
output_bias = False

net_ds = tt.practical.MLPVanilla(in_features, num_nodes, out_features, batch_norm, dropout, output_bias=output_bias)

model_ = CoxPH(net_ds, tt.optim.Adam)

In [87]:
print(type(np.asarray(x_train)))
print(type(y_train))

<class 'numpy.ndarray'>
<class 'tuple'>


In [89]:
deepsurv(data = simsurvdata(50))

NameError: name 'deepsurv' is not defined

In [88]:
batch_size = 128

# finding the best learning rate from this model
lrfinder = model_.lr_finder(np.asarray(x_train), np.asarray(y_train), batch_size, tolerance=10)
#_ = lrfinder.plot()
lrfinder.get_best_lr()

# setting the new number
model_.optimizer.set_lr(0.001)
model_.optimizer.param_groups[0]['lr']

epochs = 512

callbacks = [tt.callbacks.EarlyStopping()]
verbose = True

log = model_.fit(x_train, y_train, batch_size, epochs, callbacks, verbose,
                val_data=val, val_batch_size=batch_size)

RuntimeError: Need all tensors to have same lenght.